![Image](./resources/header.png)
<h1 align="center">Navigating the Allen Brain Observatory</h1> 
<h3 align="center">TReND CaMinA 2024</h3>

<div style="background: #ADD8E6; border-radius: 3px; padding: 10px;">

In this notebook we will use data from the Visual Coding dataset to look at how a neuron responds to visual stimulus. We will compute a "tuning curve" for a neuron's response to the drifting grating stimulus. This will reinforce how we access and work with these data.
</div>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

### Brain Observatory Setup

In [2]:
import os
import platform

# Set file location based on platform. 
platstring = platform.platform()
if ('Darwin' in platstring) or ('macOS' in platstring):
    # macOS 
    data_root = "/Volumes/TReND2024/"
elif 'Windows'  in platstring:
    # Windows (replace with the drive letter of USB drive)
    data_root = "E:/"
elif ('amzn' in platstring):
    # then on Code Ocean
    data_root = "../data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = "/media/$USERNAME/TReND2024/"

manifest_file = os.path.join(data_root,'allen-brain-observatory/visual-coding-2p/manifest.json')

`manifest_file` is a path to the manifest file.  This needs to reflect where you are storing and accessing the data. If you leave this out, a manifest file will be created in your working directory, and data will be downloaded to this location.

In [3]:
from allensdk.core.brain_observatory_cache import BrainObservatoryCache

#This instantiates the Brain Observatory Cache
boc = BrainObservatoryCache(manifest_file=manifest_file)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

<h1>Part 1: Compute the tuning curve for the drifting grating stimulus</h1>

We are going to look at the response of a single neuron, identified by its `cell specimen id`.

</div>

In [4]:
cell_id = 541513979

First get the dataset for the **experiment session** that this cell is in that has the drifting grating stimulus

In [5]:
exps = boc.get_ophys_experiments(cell_specimen_ids=[cell_id], stimuli=['drifting_gratings'])
session_id = exps[0]['id']
data_set = boc.get_ophys_experiment_data(session_id)

<div style="background: #FFF0F0; border-radius: 3px; padding: 10px;">
<p>**Exercise:** Get the DF/F traces for this session. How many neurons are in this experiment?</div>

<div style="background: #FFF0F0; border-radius: 3px; padding: 10px;">
<p>**Exercise:** Find the cell specimen index for the neuron we're looking for</div>

<div style="background: #FFF0F0; border-radius: 3px; padding: 10px;">
<p>**Exercise:** Extract the trace for our chosen neuron and call it dff_trace. </div>

Let's plot the DF/F trace of our neuron to see what it looks like.

In [ ]:
plt.figure(figsize=(14,5))
plt.plot(dff_trace)
plt.xlabel("Frames", fontsize=16)
plt.ylabel("DFF", fontsize=16)

<div style="background: #FFF0F0; border-radius: 3px; padding: 10px;">
<p>**Exercise:** Get the stimulus table for the drifting grating stimulus. Call it stim_table</div>

Let's look at the stimulus table to see what information there is. 

In [ ]:
stim_table.head()

<div style="background: #FFF0F0; border-radius: 3px; padding: 10px;">
<p>**Exercise:** What are the orientations and temporal frequencies that are used? What is the duration of each trial? How much time is between the trials?</div>

To look at the neuron's response to a given trial, let's plot the DF/F of the cell during that trial.  

For visualization purposes, let's pad the plot with ~ 1 second of the DF/F trace preceding the grating presentation and ~ 1 second after the presentation. (1 second = 30 frames.) We'll plot the response to the first trial. Print the grating direction and temporal frequency as well.

In [ ]:
plt.plot(dff_trace[stim_table.start[0]-30:stim_table.end[0]+30])


plt.axvspan(30,90, color='gray', alpha=0.3) #this shades the period when the stimulus is being presented
plt.ylabel("DF/F")
plt.xlabel("Frames")

print("Direction: ", stim_table.orientation[0])
print("Temporal frequency: ", stim_table.temporal_frequency[0])

Quantify this response by calculating the mean DF/F during the grating presentation. Only take the mean during the presentation.

In [ ]:
dff_trace[stim_table.start[0]:stim_table.end[0]].mean()

<div style="background: #FFF0F0; border-radius: 3px; padding: 10px;">
<p>**Exercise:** Repeat this (the plot and the quantification) for the next trial. </div>

To compute the tuning curve of this neuron, we need to compute this mean response to each trial of the stimulus. How many trials are there in total?

Let's do this systematically. We'll create a pandas DataFrame to hold the response of our neuron to each trial. This DataFrame needs to have the same number of trials as the stimulus table and (for now) just one column.

In [22]:
response = pd.DataFrame(columns=('orientation','temporal_frequency','trial_response'), index=stim_table.index.values) 

In [23]:
response['orientation'] = stim_table.orientation
response['temporal_frequency'] = stim_table.temporal_frequency

In [24]:
pd.options.mode.copy_on_write = True #this let's us write into the dataframe 

In [25]:
for ind,row_stim in stim_table.iterrows():
    response.loc[ind, 'trial_response'] = dff_trace[int(row_stim.start):int(row_stim.end)].mean()

Confirm that the first two trials give us the same result as we saw when we computed it individually

In [ ]:
response.head()

Let's look at these response. Plot the trial_response for all trials.

In [ ]:
plt.plot(response.trial_response, 'o')
plt.xlabel("Trial number")
plt.ylabel("Mean DFF (%)")

If we only care about one stimulus parameter, we can quickly compare the response to that parameter, say the direction. Here we will plot each grating response as a function of the grating orientation.

In [ ]:
plt.plot(response.orientation.values, response.trial_response.values, 'o')
plt.xticks(range(0,360,45))
plt.xlim(-10,325)
plt.xlabel("Direction", fontsize=16)
plt.ylabel("Mean DF/F", fontsize=16)

<div style="background: #FFF0F0; border-radius: 3px; padding: 10px;">
<p>**Exercise 2:** Repeat this plot for the temporal frequency parameter </div>

Let's compute the mean response for each orientation by averaging all of the trials for a given orientation together. To do this, you need to know what all the possible orientation values are.  Find the <b>unique</b> values that are not NaNs (eg. values that are <b>finite</b>) from either the stimulus table or the response dataframe you just made. Sort these in ascending order.

In [30]:
all_ori = np.sort(stim_table.orientation.unique())
orivals = all_ori[np.isfinite(all_ori)]
print(orivals)

[  0.  45.  90. 135. 180. 225. 270. 315.]


In [31]:
#alternate method
np.sort(stim_table.orientation.dropna().unique())

array([  0.,  45.,  90., 135., 180., 225., 270., 315.], dtype=float32)

Pandas allows us to select rows of the dataframe on a condition. For example, let's get all the trials when the orientation is 0

In [ ]:
response[response.orientation==0]

Compute and plot the mean response as a function of orientation

Make an array of length 8 to hold your results. Iterate over the orientation values, select the trials that have that orientation, and average the responses together.


<div style="background: #FFF0F0; border-radius: 3px; padding: 10px;">
<p>**Exercise 4:** Compute and plot the mean response as a function of temporal frequency for all orientations. </div>

<div style="background: #FFF0F0; border-radius: 3px; padding: 10px;">
<p>**Exercise 5:** Add errorbars to the above tuning curves. They can be standard deviation or standard error or the mean.  (Hint: `plt.errorbar` might be a useful function).
</div>

<div style="background: #FFF0F0; border-radius: 3px; padding: 10px;">
**Exercise 6:** Add a black line showing the mean response to the blank sweep (Hint 1: orientation and temporal frequency are NaN for the blank sweep condition.  Hint 2: plt.axhline might be a useful function).  </div>

<div style="background: #FFF0F0; border-radius: 3px; padding: 10px;">
<p>**Exercise 7:** So far we've been looking at each dimension separately, e.g. looking at direction tuning across all temporal frequencies. Now let's compute and plot the direction tuning curve separately for each of the 5 temporal frequencies. 
<p> What shape array do we need to hold this?
</div>

Plot the response for each direction separately

Plot a heatmap of the responses to all conditions using `plt.imshow`

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

<h1> Part 2: Compute and compare the tuning for all neurons in this experiment</h1>

Now we are going to compute the same tuning curve for all of the neurons in your experiment. 
</div>

We'll take the dff traces for all of the neurons in this experiment and compute the trial responses for each neuron. We'll put these in a DataFrame where each row is a trial (with the same index as the stimulus table) and each column is a neuron's response.

In [46]:
number_cells = dff.shape[0]
trial_responses = pd.DataFrame(index=stim_table.index.values, columns=np.array(range(number_cells)).astype(str))

for ind,row_stim in stim_table.iterrows():
    trial_responses.loc[ind] = dff[:, int(row_stim.start):int(row_stim.end)].mean(axis=1)



In [ ]:
trial_responses

Let's compute the tuning for all our neurons. We'll create an array with dimensions (8,5,number cells) with the mean response to each condition (i.e. the 2D tuning) for each neuron in this experiment

Confirm this by plotting the tuning for the neuron that we used above. 

In [ ]:
fig = plt.figure(figsize=(6,6))
plt.imshow(response[:,:,cell_index])
plt.xticks(range(5), tfvals.astype(int))
plt.yticks(range(8), orivals.astype(int))
plt.xlabel("Temporal frequency (Hz)")
plt.ylabel("Direction (deg)")
cbar = plt.colorbar()
cbar.set_label("DF/F")

<div style="background: #FFF0F0; border-radius: 3px; padding: 10px;">
<p>**Exercise** Let's look at some other neurons. Here's a list of ids from this experiment. Find the cell indices for these ids and plot their tunings.
</div>.

In [51]:
ids = [541511905, 541512490, 541512611, 541512645, 541512079, 541511403, 541511670, 541511373, 541513771, 541511385, 541512607]

<div style="background: #FFF0F0; border-radius: 3px; padding: 10px;">
<p>**Discuss:** What are some of the differences that we see among these neurons? Are there some metrics we could use to quantify them?</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

<h1> Part 3: Movie responses </h1>

Drifting gratings isn't the only stimulus presented during this session. Let's look at the response of our neuron to the natural movies.
</div>

Find what other stimuli were presented during this session.

In [ ]:
data_set.get_stimulus_epoch_table()

The DF/F traces we have are for the entire session, so we already have that. But we need the stimulus table for the natural movie.

In [55]:
stim_table_nm3 = data_set.get_stimulus_table('natural_movie_three')

In [ ]:
stim_table_nm3.head()

<div style="background: #FFF0F0; border-radius: 3px; padding: 10px;">
<p>**Exercise:** How many frames are in the natural movie? How many repeats? </div>

Let's look at the first and the last frame of the movie. Get the stimulus template for this movie and plot the first and the last frame of the movie.

In [59]:
template = data_set.get_stimulus_template('natural_movie_three')

In [60]:
template.shape

(3600, 304, 608)

In [ ]:
plt.imshow(template[0,:,:], cmap='gray')

In [ ]:
plt.imshow(template[-1,:,:], cmap='gray')

Compute the mean response for each neuron to the movie.  

Plot the mean movie response for the cell you analyzed above

In [ ]:
plt.plot(response_nm3[cell_index,:])

<div style="background: #FFF0F0; border-radius: 3px; padding: 10px;">
<p>**Exercise 9:** Plot these movie responses for some of the other neurons that we looked at above.
</div>.